## Resumen del Notebook: etiquetas_parte_1

### Objetivo:
Analizar reseñas de clientes de hoteles para categorizarlas en base a su contenido, utilizando técnicas de NLP.

### Proceso:
Definición de Categorías:

Establecimos categorías basadas en quejas comunes de clientes de hoteles.
Las categorías se expandieron para abordar aspectos tanto negativos como positivos.
Se usaron prefijos "n_" para categorías negativas y "p_" para categorías positivas.
Preparación de Datos:

Tomamos una muestra aleatoria del 2% de las reseñas para hacer el análisis y modelado iniciales.
Convertimos las reseñas a embeddings utilizando FastText.

Similitud de Coseno:

Calculamos la similitud de coseno entre cada reseña (en forma de embedding) y las palabras clave de cada categoría para determinar la categoría más similar para cada reseña.
Asignamos la categoría con la similitud más alta a cada reseña.

Visualización:

Mostramos un subconjunto del DataFrame para revisar las reseñas y sus categorías asignadas.

### Posibles Pasos Futuros:
Utilizar las categorías asignadas a la muestra del 2% para entrenar un modelo de clasificación supervisado.
Usar dicho modelo para predecir las categorías de las reseñas restantes en el conjunto de datos.
Refinar las categorías o palabras clave según sea necesario, basándose en la retroalimentación y los resultados obtenidos.

In [1]:
import pandas as pd

# Cargar el DataFrame
df = pd.read_parquet("tabla_final_hoteles_google_yelp.parquet")

# Filtrar el DataFrame para mantener solo las filas donde 'comentario' no es una cadena vacía
df = df[df['comentario'] != '']

# Tomar una muestra aleatoria del 2%
df_sample = df.sample(frac=0.02, random_state=42)


In [2]:
df_sample.shape

(3333, 5)

In [3]:
import re

df_sample['comentario'] = df_sample['comentario'].str.lower()  # Convertir a minúsculas
df_sample['comentario'] = df_sample['comentario'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))  # Eliminar signos de puntuación


In [4]:
#!pip install fasttext

In [5]:
import fasttext

df_sample['comentario'].to_csv("fasttext_input_sample.txt", index=False, header=None)

model = fasttext.train_unsupervised("fasttext_input_sample.txt", model="skipgram")


Read 0M words
Number of words:  4227
Number of labels: 0
Progress: 100.0% words/sec/thread:   27958 lr:  0.000000 avg.loss:  2.546410 ETA:   0h 0m 0s


In [6]:
def get_vector(text, model):
    # Eliminar los saltos de línea del texto
    clean_text = text.replace('\n', ' ')
    return model.get_sentence_vector(clean_text)

df_sample['embedding'] = df_sample['comentario'].apply(lambda x: get_vector(x, model))


In [7]:
nan_in_embedding = df_sample['embedding'].apply(lambda x: any(pd.isna(x))).sum()
print(f'Número de listas embedding con NaNs: {nan_in_embedding}')

Número de listas embedding con NaNs: 0


In [8]:
clasificaciones = {
    "n_Higiene": ["smell of smoking rooms", "bed bugs", "black mold", "musty smells", "cigarette burns", "bug bites", "blood stains", "trash", "dirty carpet", "cockroaches", "dust", "stains", "dirty linens"],
    "n_Mantenimiento": ["broken or dirty toilets seats", "broken hair dryers", "water damage", "cold water", "hot air", "cold air", "faulty lighting", "broken furniture", "chipped paint"],
    "n_Servicio": ["poor service", "rude staff", "third party booking services", "wrong rooms", "slow check-in", "incorrect charges", "unresponsive staff"],
    "n_Suministros": ["not enough toilet paper", "no toilet paper", "lack of towels", "empty minibar"], 
    "n_Aislamiento": ["thin walls", "noisy neighbors", "street noise", "creaky floors"],
    "n_Confort": ["uncomfortable bed", "cramped room", "outdated decor", "unpleasant odors", "hard mattress"],
    "n_Limpieza": ["dirty room", "poor maintenance", "stained linens", "clogged drains", "unclean bathroom"],
    "n_Atención al cliente": ["unfriendly staff", "unhelpful staff", "indifferent service", "cold reception", "inattentive staff"],
    "n_Facilidades": ["lacking amenities", "dirty pool", "broken gym equipment", "poor Wi-Fi connection", "limited parking"],
    "n_Ubicación": ["bad location", "too far from downtown", "far from attractions", "inconvenient location", "noisy street"],
    "n_Valor": ["overpriced", "not worth the price", "expensive for what you get", "hidden charges"],
    "n_Experiencia gastronómica": ["bad breakfast", "unpalatable meals", "limited buffet choices", "stale food", "poor restaurant service"],
    "n_Ambiente": ["dated decor", "stale atmosphere", "unpleasant ambiance", "bad views", "loud interiors"],
    "p_Confort": ["comfortable bed", "spacious room", "modern decor", "pleasant aroma", "soft mattress"],
    "p_Limpieza": ["clean room", "well-maintained", "fresh linens", "clean drains", "spotless bathroom"],
    "p_Atención al cliente": ["friendly staff", "helpful staff", "warm reception", "attentive staff"],
    "p_Facilidades": ["excellent amenities", "clean pool", "modern gym equipment", "strong Wi-Fi connection", "ample parking"],
    "p_Ubicación": ["great location", "close to downtown", "near attractions", "convenient location", "quiet street"],
    "p_Valor": ["good value", "worth the price", "budget-friendly", "no hidden charges"],
    "p_Experiencia gastronómica": ["delicious breakfast", "tasty meals", "varied buffet choices", "fresh food", "excellent restaurant service"],
    "p_Ambiente": ["modern decor", "vibrant atmosphere", "pleasant ambiance", "beautiful views", "quiet interiors"]
}


clasificaciones_embeddings = {key: [model.get_sentence_vector(word) for word in words] for key, words in clasificaciones.items()}


In [9]:
#!pip install scikit-learn

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

def get_most_similar_category(review_embedding, clasificaciones_embeddings):
    max_similarity = -float('inf')
    best_category = None
    
    for category, embeddings in clasificaciones_embeddings.items():
        for word_embedding in embeddings:
            similarity = cosine_similarity([review_embedding], [word_embedding])[0][0]
            if similarity > max_similarity:
                max_similarity = similarity
                best_category = category
    return best_category


In [11]:
df_sample['categoria'] = df_sample['embedding'].apply(lambda x: get_most_similar_category(x, clasificaciones_embeddings))


In [12]:
df_sample

,business_id,rating,comentario,ciudad,estado,embedding,categoria
26421,Kxg8HBv9X2kfz_JQuZzmDw,3.0,the area is kind of sketchy and when youre fir...,New Orleans,Louisiana,"[0.035103217, -0.080136076, -0.038549528, 0.03...",n_Ambiente
20950,1I_qcjb-UuAiA7kwT_xPCA,3.0,pros\nawesome indoor and outdoor pool and ind...,Nashville,Tennessee,"[0.040991627, -0.056416266, -0.05178736, 0.023...",n_Ambiente
176884,xeRbOKTcZZvk6BI_YQ9Ipw,1.0,terrible service everyone there has a bad atti...,New Orleans,Louisiana,"[0.011211632, -0.054489277, -0.06136544, 0.023...",n_Confort
117677,r6_wKaVLLnYT31oGnxFuHA,5.0,great location close to bourbon street minute...,New Orleans,Louisiana,"[0.04921449, -0.07848664, -0.042117387, 0.0021...",p_Confort
156460,OZnYatmpbZwbtaFTJlYVhg,1.0,every time i get to a room the first thing i d...,Tampa,Florida,"[0.015249598, -0.050747488, -0.04434709, 0.036...",n_Confort
...,...,...,...,...,...,...,...
91923,8C0jJ6AmcDZGscuABtZRdQ,5.0,maybe its rated as a three star hotel if it i...,Tampa,Florida,"[0.045517575, -0.07559743, -0.042018212, 0.027...",n_Ambiente
140228,0x88f5ac62bc6f21cb:0x5ae7bf2041b49721,5.0,clean,Decatur,Georgia,"[0.105572395, -0.11329313, -0.1877839, 0.03474...",p_Facilidades
1109,2tzM3YrsqPa4qPVsfn3-tA,2.0,checked out at am and they had my room cleaned...,Indianapolis,Indiana,"[0.012525278, -0.04236787, -0.018842155, 0.042...",n_Confort
64107,fcohLhqJ5GHOSoJfCcrLUQ,4.0,awesome hotel we got upgraded since it was our...,Tampa,Florida,"[0.07335173, -0.09491684, -0.03958444, 0.04021...",n_Ambiente


In [13]:
df_subset = df_sample[['comentario', 'categoria']]
df_subset


,comentario,categoria
26421,the area is kind of sketchy and when youre fir...,n_Ambiente
20950,pros\nawesome indoor and outdoor pool and ind...,n_Ambiente
176884,terrible service everyone there has a bad atti...,n_Confort
117677,great location close to bourbon street minute...,p_Confort
156460,every time i get to a room the first thing i d...,n_Confort
...,...,...
91923,maybe its rated as a three star hotel if it i...,n_Ambiente
140228,clean,p_Facilidades
1109,checked out at am and they had my room cleaned...,n_Confort
64107,awesome hotel we got upgraded since it was our...,n_Ambiente


In [14]:
df_not_sampled = df.drop(df_sample.index)

In [15]:
df_not_sampled.to_csv('resto_1.csv', index=False)

In [16]:
# Obtener el conteo de cada categoría
conteo_categorias = df_sample['categoria'].value_counts()

# Imprimir el conteo de cada categoría
conteo_categorias


n_Ambiente                    1826
n_Confort                      578
n_Valor                        331
p_Ambiente                     168
n_Suministros                   76
n_Servicio                      70
n_Ubicación                     59
n_Experiencia gastronómica      56
p_Valor                         37
n_Atención al cliente           34
p_Confort                       18
p_Facilidades                   18
n_Higiene                       16
p_Limpieza                      12
n_Mantenimiento                 12
p_Atención al cliente            8
p_Ubicación                      7
p_Experiencia gastronómica       3
n_Facilidades                    2
n_Limpieza                       1
n_Aislamiento                    1
Name: categoria, dtype: int64

In [17]:
#!pip install unidecode

In [18]:
import unidecode

def remove_accents(input_str):
    return unidecode.unidecode(input_str)

# Aplica la función para quitar acentos en la columna 'categoria'
df_sample['categoria'] = df_sample['categoria'].apply(remove_accents)

# Verificar las categorías después de quitar los acentos
print(df_sample['categoria'].value_counts())


n_Ambiente                    1826
n_Confort                      578
n_Valor                        331
p_Ambiente                     168
n_Suministros                   76
n_Servicio                      70
n_Ubicacion                     59
n_Experiencia gastronomica      56
p_Valor                         37
n_Atencion al cliente           34
p_Confort                       18
p_Facilidades                   18
n_Higiene                       16
p_Limpieza                      12
n_Mantenimiento                 12
p_Atencion al cliente            8
p_Ubicacion                      7
p_Experiencia gastronomica       3
n_Facilidades                    2
n_Limpieza                       1
n_Aislamiento                    1
Name: categoria, dtype: int64


In [19]:
df_sample['categoria'] = df_sample['categoria'].str.replace(' ', '_')

# Verificar las categorías después de reemplazar los espacios
print(df_sample['categoria'].value_counts())


n_Ambiente                    1826
n_Confort                      578
n_Valor                        331
p_Ambiente                     168
n_Suministros                   76
n_Servicio                      70
n_Ubicacion                     59
n_Experiencia_gastronomica      56
p_Valor                         37
n_Atencion_al_cliente           34
p_Confort                       18
p_Facilidades                   18
n_Higiene                       16
p_Limpieza                      12
n_Mantenimiento                 12
p_Atencion_al_cliente            8
p_Ubicacion                      7
p_Experiencia_gastronomica       3
n_Facilidades                    2
n_Limpieza                       1
n_Aislamiento                    1
Name: categoria, dtype: int64


In [20]:
df_export = df_sample[['comentario', 'categoria']].copy()
df_export['comentario'] = df_export['comentario'].astype(str)
df_export.to_parquet('muestra_etiquetas_1.parquet', index=False)


In [21]:
s1 = pd.read_parquet("muestra_etiquetas_1.parquet")

In [22]:
initial_nan_count = s1['comentario'].isna().sum()
print("Número inicial de valores NaN en 'comentario':", initial_nan_count)


Número inicial de valores NaN en 'comentario': 0


In [23]:
initial_nan_count = df_sample['comentario'].isna().sum()
print("Número inicial de valores NaN en 'comentario':", initial_nan_count)


Número inicial de valores NaN en 'comentario': 0


In [24]:
initial_nan_count = df_export['comentario'].isna().sum()
print("Número inicial de valores NaN en 'comentario':", initial_nan_count)


Número inicial de valores NaN en 'comentario': 0


In [25]:
empty_strings_count = (df_sample['comentario'] == '').sum()
print("Número de registros con cadena vacía en 'comentario':", empty_strings_count)


Número de registros con cadena vacía en 'comentario': 0
